In [30]:
from matrix import *
import numpy as np
import math
import random
from typing import List
random.seed(57)

def generate_matrix(rows, cols, min_val, max_val):
    return save_matrix([[random.randint(min_val, max_val) for i in range(cols)] for j in range(rows)])

# Easy 1: Метод Гаусса

In [31]:
def gauss_solver(A: Matrix, b: Matrix, tol: float = 1e-5) -> List[Matrix]:
    if A.rows != b.rows or b.columns != 1:
        raise ValueError("Размеры A и b не согласованы")

    n, m = A.rows, A.columns
    # преобразуем в плотный формат
    classic_A = get_classic_format(A)
    classic_b = get_classic_format(b)

    Ab = [row.copy() + [classic_b[i][0]] for i, row in enumerate(classic_A)]

    rank = 0
    # Прямой ход Гаусса
    for col in range(m):
        max_row = rank
        for row in range(rank + 1, n):
            if abs(Ab[row][col]) > abs(Ab[max_row][col]):
                max_row = row

        # Если максимальный элемент равен нулю, скипаем столбец
        if abs(Ab[max_row][col]) < tol:
            continue

        # Меняем строки местами
        Ab[rank], Ab[max_row] = Ab[max_row], Ab[rank]

        # Нормализация строки
        pivot = Ab[rank][col]
        for j in range(col, m + 1):
            Ab[rank][j] /= pivot

        # Исключаем переменную из других строк
        for i in range(n):
            if i != rank and abs(Ab[i][col]) > tol:
                factor = Ab[i][col]
                for j in range(col, m + 1):
                    Ab[i][j] -= factor * Ab[rank][j]

        rank += 1
        if rank == n:
            break

    # Проверяем совместность
    for i in range(rank, n):
        if abs(Ab[i][m]) > tol:
            raise ValueError("Система несовместна")

    # Обратный ход Гаусса
    solutions = []
    free_vars = []
    lead_vars = [-1] * m

    # Определяем свободные переменные
    for i in range(rank):
        for j in range(m):
            if abs(Ab[i][j]) > tol:
                lead_vars[j] = i
                break

    for j in range(m):
        if lead_vars[j] == -1:
            free_vars.append(j)

    # Если свободных нет - единственное решение
    if not free_vars:
        solution = [[0.0] for _ in range(m)]
        for i in range(rank):
            for j in range(m):
                if abs(Ab[i][j]) > tol:
                    solution[j][0] = Ab[i][m]
                    break

        sol = [(i, 0, solution[i][0]) for i in range(m) if abs(solution[i][0]) > tol]
        return [Matrix(m, 1, sol)]

    # Если свободные есть - строим базис
    for free in free_vars:
        vec = [[0.0] for _ in range(m)]
        vec[free][0] = 1.0

        for i in range(rank):
            for j in range(m):
                if abs(Ab[i][j]) > tol:
                    sum_ax = 0.0
                    for k in range(j + 1, m):
                        sum_ax += Ab[i][k] * vec[k][0]
                    vec[j][0] = Ab[i][m] - sum_ax
                    break

        basis = [(i, 0, vec[i][0]) for i in range(m) if abs(vec[i][0]) > tol]
        solutions.append(Matrix(m, 1, basis))

    return solutions

In [32]:
# Пример системы уравнений:
# 1x + 2y = 5
# 3x + 4y = 11

A = save_matrix([[1, 2], [3, 4]])
b = save_matrix([[5], [11]])

solutions = gauss_solver(A, b)
if len(solutions) == 1:
    print("Единственное решение:")
    print(solutions[0])
else:
    print(f"Бесконечно много решений. Базис ФСР (размерность {len(solutions)}):")
    for i, sol in enumerate(solutions):
        print(f"Решение {i+1}:")
        print(sol)

Единственное решение:
1.0
2.0


# Easy 2: Центрирование данных
$$ X_{centered} = X - X_{mean} $$

In [33]:
def get_means(X: Matrix) -> List[float]:
    rows = X.rows
    cols = X.columns
    means = [0.0] * cols

    for col in range(1, cols + 1):
        for row in range(1, rows + 1):
            means[col-1] += X.get(row, col)
        means[col-1] /= rows

    return means

def center_data(X: Matrix) -> Matrix:
    means = get_means(X)
    centered_data = []

    for row in range(1, X.rows + 1):
        for col in range(1, X.columns + 1):
            val = X.get(row, col)
            centered_val = val - means[col-1]
            centered_data.append((row-1, col-1, centered_val))

    return Matrix(X.rows, X.columns, centered_data)

Сравнение с NumPy:

In [34]:
X = generate_matrix(3, 3, -3, 3)

print("Исходная матрица:")
print(X)

X_centered = center_data(X)
print("\nНаше центрирование:")
print(X_centered)

X_np = np.array(get_classic_format(X))
centered_np = X_np - X_np.mean(axis=0)

print("\nNumPy центрирование:")
print(centered_np)


Исходная матрица:
-3.0 -1.0 1.0
1.0 -3.0 -2.0
1.0 -1.0 0

Наше центрирование:
-2.6666666666666665 0.6666666666666667 1.3333333333333333
1.3333333333333333 -1.3333333333333333 -1.6666666666666667
1.3333333333333333 0.6666666666666667 0.3333333333333333

NumPy центрирование:
[[-2.66666667  0.66666667  1.33333333]
 [ 1.33333333 -1.33333333 -1.66666667]
 [ 1.33333333  0.66666667  0.33333333]]


# Easy 3: матрица ковариаций
$$ C = \frac{1}{n-1}X^TX $$

In [35]:
def covariance_matrix(X_centered: Matrix) -> Matrix:
    n = X_centered.rows
    p = X_centered.columns
    X = get_classic_format(X_centered)

    cov_data = []
    for i in range(p):
        for j in range(p):
            cov = sum(X[row][i] * X[row][j] for row in range(n)) / (n - 1)
            cov_data.append((i, j, cov))

    return Matrix(p, p, cov_data)

Сравнение с NumPy:

In [36]:
# Тестовые данные
data = [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]
X = save_matrix(data)


X_centered = center_data(X)
cov = covariance_matrix(X_centered)

X_np = np.array(data)
cov_numpy = np.cov(X_np, rowvar=False, bias=False)


print("Наш результат:")
print(cov)
print("\nNumPy результат:")
print(cov_numpy)


Наш результат:
9.0 9.0 9.0
9.0 9.0 9.0
9.0 9.0 9.0

NumPy результат:
[[9. 9. 9.]
 [9. 9. 9.]
 [9. 9. 9.]]


# Normal 1: Собственные значения

In [37]:
# поиск корня бисекцией
def root_search(f, a, b, tol=1e-6):
    fa, fb = f(a), f(b)
    if fa * fb > 0:
        return None

    while (b-a) > tol:
        c = (a + b) / 2
        fc = f(c)
        if abs(fc) < tol:
            return c
        if fa * fc < 0:
            b, fb = c, fc
        else:
            a, fa = c, fc
    return (a + b) / 2

# поиск экстремума бисекцией
def extremum_search(f, a0, b0, epsilon, delta=1e-10):
    a = a0
    b = b0
    ans = (a + b) / 2

    while abs(b - a) > epsilon:
        yk = (a + b - delta) / 2
        zk = (a + b + delta) / 2

        if f(yk) <= f(zk):
            b = zk
        else:
            a = yk

        ans = (a + b) / 2
    return ans

In [38]:
# основная функция. поиск собственных значений
def find_eigenvalues(C_matrix: 'Matrix', tol: float = 1e-6) -> List[float]:
    n = C_matrix.rows
    C = get_classic_format(C_matrix)

# обычный характеристический полином
    def characteristic_poly(lam):
        mat = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
        #return determinant(mat)
        # чтоб оперативней считалось, потом закомментить
        return np.linalg.det(mat)

    # модуль полинома для корней чётной кратности
    def poly_abs(lam):
        mat = [[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)]
        #return abs(determinant(mat))
        # чтоб оперативней считалось, потом закомментить
        return abs(np.linalg.det(mat))

    # Вычисляем интервал поиска (см теорему Гершгорина)
    Gershgorin_intervals = []
    for i in range(n):
        radius = sum(abs(C[i][j]) for j in range(n) if j != i)
        center = C[i][i]
        Gershgorin_intervals.append((center - radius, center + radius))

    lower = min(interval[0] for interval in Gershgorin_intervals)
    upper = max(interval[1] for interval in Gershgorin_intervals)

    # Расширяем интервал, ну чтобы понадёжнее
    lower -= 3
    upper += 3
    count = max(100, 10 * n)

    # непосредственно поиск корней
    def find_roots(f, fabs, a, b, tol, count):
        roots = []
        # дробим [a,b] на мелкие подотрезки
        step = (b - a) / count

        x_prev = a
        f_prev = f(x_prev)

        # пробегаем по каждому маленькому отрезку
        for i in range(1, count+1):
            x = a + i * step
            fx = f(x)

            # если попали в корень
            if abs(fx) < tol:
                roots.append(x)
            # если функция меняет знак, ищем корень
            # если не меняет, смотрим экстремум для функции-модуля.
            if f_prev * fx < 0:
                roots.append(root_search(f, x_prev, x, tol))
            else:
                exst = extremum_search(fabs, x_prev, x, tol)
                # если при поиске экстремума «скатились» близко к нулю, то всё ок
                if abs(f(exst)) < 0.0001:
                  roots.append(exst)

            x_prev = x
            f_prev = fx

        # убираем дубликаты (близкие корни)
        unique_roots = [roots[0]]
        for i in range(1,len(sorted(roots))):
            if not unique_roots or (roots[i] - roots[i-1] > 2*step):
                unique_roots.append(roots[i])

        return unique_roots

    eigenvalues = find_roots(characteristic_poly, poly_abs, lower, upper, tol, count)
    return sorted(eigenvalues)[::-1]

In [39]:
C = generate_matrix(3, 3, 1, 2)
print("Исходная матрица:")
print(C)

eigenvalues = find_eigenvalues(C)
print("\nНайденные собственные значения:")
for i, val in enumerate(eigenvalues, 1):
    print(f"lambda{i} = {val:.6f}")

np_matrix = get_classic_format(C)
np_eigenvalues = np.linalg.eigvals(np_matrix)
print("\nПроверка с numpy:")
for i, val in enumerate(sorted(np_eigenvalues, reverse=True), 1):
    print(f"lambda{i} = {val:.6f}")

Исходная матрица:
2.0 2.0 2.0
1.0 1.0 2.0
2.0 2.0 1.0

Найденные собственные значения:
lambda1 = 5.000000
lambda2 = 0.000000
lambda3 = -1.000000

Проверка с numpy:
lambda1 = 5.000000
lambda2 = -0.000000
lambda3 = -1.000000


# Normal 2: Собственные векторы

In [40]:
def normalize_vector(vect: Matrix) -> Matrix:
    vector = get_classic_format(vect)
    norm = math.sqrt(sum(x[0] ** 2 for x in vector))
    return save_matrix([[x[0] / norm] for x in vector])

def find_eigenvectors(C_matrix: Matrix, eigenvalues: List[float], tol = 1e-2) -> List[Matrix]:
    n = C_matrix.rows
    vectors_list = []

    C = get_classic_format(C_matrix)
    for lam in eigenvalues:
        C_lam = save_matrix([[C[i][j] - (lam if i == j else 0) for j in range(n)] for i in range(n)])
        vectors_lam = gauss_solver(C_lam, save_matrix([[0]]*n), tol)
        vectors_list += vectors_lam

    normalize_list = []
    for vectors in vectors_list:
        normalize_list.append(normalize_vector(vectors))
    return normalize_list

Сравнение с NumPy:

In [41]:
C = save_matrix([[-3, 18, 9, 50, 80],
               [7, -86, -25, -15, 94],
               [-65, 6, -66, 18, 67],
               [22, -1, -39, -41, 87],
               [-85, -3, 26, 2, 22]])

X_centered = center_data(C)
covar = covariance_matrix(X_centered)
vals = find_eigenvalues(covar)
vects = find_eigenvectors(covar, vals, 1e-2)

np_covar = np.cov(get_classic_format(X_centered), rowvar=False)
np_vals, np_vects = np.linalg.eig(np_covar)

sorted_indices = np.argsort(np_vals)[::-1]
np_vals_sorted = np_vals[sorted_indices]
np_vects_sorted = np_vects[:, sorted_indices]

# Сравнение
print("Собственные значения:")
print("Наши:", vals)
print("NumPy:", np_vals_sorted)

print("Наши:", np_vects_sorted)
print("NumPy:", vects)



Собственные значения:
Наши: [3726.2397330184986, 1519.9276982215451, 1311.9728817286677, 751.4596866568662, -1.715086294982136e-07]
NumPy: [ 3.72623973e+03  1.51992770e+03  1.31197288e+03  7.51459687e+02
 -1.81177772e-13]
Наши: [[-0.68384366  0.43137162 -0.37033079 -0.17831082  0.42111368]
 [ 0.42365012  0.74928182  0.17719889 -0.46200595 -0.11936664]
 [ 0.27550092 -0.13694744 -0.87798761 -0.23306736 -0.28312896]
 [ 0.31935569  0.40121405 -0.24540173  0.79084645  0.22666961]
 [-0.4183176   0.26975538  0.0192721   0.27385626 -0.82272338]]
NumPy: [0.6838436577211879
-0.4236501164027639
-0.27550092389826764
-0.3193556887865307
0.41831760139553414, 0.43137162279175784
0.7492818176077534
-0.13694744114231064
0.40121405016732736
0.26975538023425893, -0.3703307906465555
0.1771988900753271
-0.8779876066034304
-0.2454017271334847
0.019272099674185326, -0.1783108244042088
-0.4620059473645255
-0.23306735863258163
0.7908464515463494
0.2738562595900961, -0.42111367842028113
0.11936663973317065
0.28

# Normal 3: Доля объяснённой дисперсии

In [42]:
def explained_variance_ratio(eigenvalues: List[float], k: int) -> float:
    return sum(eigenvalues[:k]) / sum(eigenvalues)

Сравнение с NumPy:

In [43]:
C = save_matrix([[-3, 18, 9, 50, 80],
               [7, -86, -25, -15, 94],
               [-65, 6, -66, 18, 67],
               [22, -1, -39, -41, 87],
               [-85, -3, 26, 2, 22]])

X_centered = center_data(C)
covar = covariance_matrix(X_centered)
vals = find_eigenvalues(covar)


np_covar = np.cov(get_classic_format(X_centered), rowvar=False)
np_vals = np.linalg.eigvals(np_covar)
np_vals_sorted = sorted(np_vals, reverse=True)

for k in [1, 3, 5]:
    our_ratio = explained_variance_ratio(vals, k)

    np_ratio = sum(np_vals_sorted[:k]) / sum(np_vals_sorted)

    print(f"k={k}:")
    print(f"Наш результат: {our_ratio:.6f}")
    print(f"NumPy результат: {np_ratio:.6f}\n")

k=1:
Наш результат: 0.509773
NumPy результат: 0.509773

k=3:
Наш результат: 0.897196
NumPy результат: 0.897196

k=5:
Наш результат: 1.000000
NumPy результат: 1.000000

